In [1]:
!pip install -U keras-tuner #for performing hyperparameter tuning
!pip install --upgrade tensorflow jax jaxlib #To ensure that we have the latest versions of TensorFlow and JAX installed.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.1/86.1 MB 9.5 MB/s eta 0:00:00
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.4.33
    Uninstalling jaxlib-0.4.33:
      Successfully uninstalled jaxlib-0.4.33
  Attempting uninstall: jax
    Found existing installation: jax 0.4.33
    Uninstalling jax-0.4.33:
      Successfully uninstalled jax-0.4.33


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
import tensorflow as tf
from tensorflow.keras.layers import Dense
import keras_tuner as kf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

In [3]:
df = pd.read_csv('/content/Alphabets_data.csv')

In [4]:
df

,letter,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,D,2,2,3,3,2,7,7,7,6,6,6,4,2,8,3,7
19996,C,7,10,8,8,4,4,8,6,9,12,9,13,2,9,3,7
19997,T,6,9,6,7,5,6,11,3,7,11,9,5,2,12,2,4
19998,S,2,3,4,2,1,8,7,2,6,10,6,8,1,9,5,8


In [5]:
df.head()

,letter,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10


In [6]:
df.tail()

,letter,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
19995,D,2,2,3,3,2,7,7,7,6,6,6,4,2,8,3,7
19996,C,7,10,8,8,4,4,8,6,9,12,9,13,2,9,3,7
19997,T,6,9,6,7,5,6,11,3,7,11,9,5,2,12,2,4
19998,S,2,3,4,2,1,8,7,2,6,10,6,8,1,9,5,8
19999,A,4,9,6,6,2,9,5,3,1,8,1,8,2,7,2,8


In [7]:
df.describe()

,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
count,20000.000000,20000.000000,20000.000000,20000.00000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.00000,20000.000000,20000.000000,20000.000000,20000.000000,20000.00000
mean,4.023550,7.035500,5.121850,5.37245,3.505850,6.897600,7.500450,4.628600,5.178650,8.282050,6.45400,7.929000,3.046100,8.338850,3.691750,7.80120
std,1.913212,3.304555,2.014573,2.26139,2.190458,2.026035,2.325354,2.699968,2.380823,2.488475,2.63107,2.080619,2.332541,1.546722,2.567073,1.61747
min,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,3.000000,5.000000,4.000000,4.00000,2.000000,6.000000,6.000000,3.000000,4.000000,7.000000,5.00000,7.000000,1.000000,8.000000,2.000000,7.00000
50%,4.000000,7.000000,5.000000,6.00000,3.000000,7.000000,7.000000,4.000000,5.000000,8.000000,6.00000,8.000000,3.000000,8.000000,3.000000,8.00000
75%,5.000000,9.000000,6.000000,7.00000,5.000000,8.000000,9.000000,6.000000,7.000000,10.000000,8.00000,9.000000,4.000000,9.000000,5.000000,9.00000
max,15.000000,15.000000,15.000000,15.00000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.00000,15.000000,15.000000,15.000000,15.000000,15.00000


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   letter  20000 non-null  object
 1   xbox    20000 non-null  int64 
 2   ybox    20000 non-null  int64 
 3   width   20000 non-null  int64 
 4   height  20000 non-null  int64 
 5   onpix   20000 non-null  int64 
 6   xbar    20000 non-null  int64 
 7   ybar    20000 non-null  int64 
 8   x2bar   20000 non-null  int64 
 9   y2bar   20000 non-null  int64 
 10  xybar   20000 non-null  int64 
 11  x2ybar  20000 non-null  int64 
 12  xy2bar  20000 non-null  int64 
 13  xedge   20000 non-null  int64 
 14  xedgey  20000 non-null  int64 
 15  yedge   20000 non-null  int64 
 16  yedgex  20000 non-null  int64 
dtypes: int64(16), object(1)
memory usage: 2.6+ MB


In [9]:
df.shape

(20000, 17)

In [10]:
X = df.drop('letter', axis=1)
y = df['letter']

In [11]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   xbox    20000 non-null  int64
 1   ybox    20000 non-null  int64
 2   width   20000 non-null  int64
 3   height  20000 non-null  int64
 4   onpix   20000 non-null  int64
 5   xbar    20000 non-null  int64
 6   ybar    20000 non-null  int64
 7   x2bar   20000 non-null  int64
 8   y2bar   20000 non-null  int64
 9   xybar   20000 non-null  int64
 10  x2ybar  20000 non-null  int64
 11  xy2bar  20000 non-null  int64
 12  xedge   20000 non-null  int64
 13  xedgey  20000 non-null  int64
 14  yedge   20000 non-null  int64
 15  yedgex  20000 non-null  int64
dtypes: int64(16)
memory usage: 2.4 MB


In [12]:
y.info()

<class 'pandas.core.series.Series'>
RangeIndex: 20000 entries, 0 to 19999
Series name: letter
Non-Null Count  Dtype 
--------------  ----- 
20000 non-null  object
dtypes: object(1)
memory usage: 156.4+ KB


Data Preprocessing:

In [13]:
X.isnull().sum()

,0
xbox,0
ybox,0
width,0
height,0
onpix,0
xbar,0
ybar,0
x2bar,0
y2bar,0
xybar,0


In [14]:
y.isnull().sum()

0

In [15]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_encoded = le.fit_transform(y)

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded,test_size=0.2, random_state=42)

In [19]:
X_train

array([[-0.01291349, -0.01275171, -0.06013429, ..., -0.21675086,
         0.12044468,  0.12215017],
       [-0.01291349, -0.01275171,  0.43581861, ..., -0.21675086,
         0.12044468, -0.49633169],
       [-0.53599158, -0.618174  , -0.55608718, ...,  0.43123377,
        -0.65897489, -1.11481354],
       ...,
       [-0.53599158, -0.92088514,  0.43581861, ..., -0.86473548,
        -0.2692651 ,  0.74063202],
       [-0.53599158,  0.28995944, -0.55608718, ..., -0.21675086,
        -1.04868467, -0.49633169],
       [-1.58214775, -1.22359629, -1.54799298, ..., -0.86473548,
        -1.43839445, -0.49633169]])

In [20]:
y_train

array([ 4,  4, 13, ..., 23,  8,  9])

In [18]:
# Feature scaling

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

ANN Model Implementation:

In [22]:
ann = Sequential()

ann.add(Dense(units = 10, activation = 'relu'))# hidden layer
ann.add(Dense(units = 1, activation = 'sigmoid'))# output layer

ann.compile(optimizer = 'adadelta', loss = 'binary_crossentropy', metrics =['accuracy'])

ann.fit(X_train,y_train,epochs = 100)

Epoch 1/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.0520 - loss: 3.5536
Epoch 2/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0527 - loss: 3.1676
Epoch 3/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0553 - loss: 3.2210
Epoch 4/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0552 - loss: 3.1083
Epoch 5/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0512 - loss: 3.0686
Epoch 6/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0528 - loss: 2.9146
Epoch 7/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0556 - loss: 2.6231
Epoch 8/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0528 - loss: 2.5691
Epoch 9/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0524 - loss: 2.2421
Epoch 10/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0513 - loss: 2.1993
Epoch 11/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0567 - loss: 2.0656
Epoch 12/100
500/500 ━━━━━━━━━━━━━━━━━━━━

In [23]:
ypred = ann.predict(X_test)
ypred = ypred > 0.5
ypred

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


array([[ True],
       [ True],
       [ True],
       ...,
       [ True],
       [ True],
       [ True]])

In [24]:
print(classification_report(y_test, ypred))

              precision    recall  f1-score   support

           0       0.07      0.28      0.12       149
           1       0.04      0.95      0.08       153
           2       0.00      0.00      0.00       137
           3       0.00      0.00      0.00       156
           4       0.00      0.00      0.00       141
           5       0.00      0.00      0.00       140
           6       0.00      0.00      0.00       160
           7       0.00      0.00      0.00       144
           8       0.00      0.00      0.00       146
           9       0.00      0.00      0.00       149
          10       0.00      0.00      0.00       130
          11       0.00      0.00      0.00       155
          12       0.00      0.00      0.00       168
          13       0.00      0.00      0.00       151
          14       0.00      0.00      0.00       145
          15       0.00      0.00      0.00       173
          16       0.00      0.00      0.00       166
          17       0.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [25]:
ann.evaluate(X_train,y_train)

500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0465 - loss: -12.0830


[-11.927816390991211, 0.04612499848008156]


Hyperparameter tuning

In [41]:
print(X_train.shape)
print(X_test.shape)

(16000, 16)
(4000, 16)


In [44]:
X_train = X_train[:, :13]
X_test = X_test[:, :13]


In [26]:
list(range(8,128,8))

[8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96, 104, 112, 120]

In [45]:
def hyper(hp):
    model = Sequential()

#Add input layer
    model.add(Dense(hp.Int('units0', min_value=8, max_value=128, step=8),
                    activation=hp.Choice('activation0', values=['relu', 'tanh', 'sigmoid']),
                    input_dim=13))
#Add additional hidden layers
    for i in range(1, hp.Int('num_layers', min_value=1, max_value=10)):
        model.add(Dense(hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
                        activation=hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid'])))
#Add output layer
    model.add(Dense(units=1, activation='sigmoid'))

#Compile the model
    model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop', 'sgd']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model


In [35]:
tuner = kf.RandomSearch(hyper,
                        objective = 'val_accuracy',
                        max_trials = 3)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [46]:
tuner.search(X_train,y_train,epochs = 3, validation_data = (X_test,y_test))

Trial 3 Complete [00h 08m 11s]
val_accuracy: 0.03824999928474426

Best val_accuracy So Far: 0.03824999928474426
Total elapsed time: 00h 08m 15s


In [47]:
tuner.get_best_hyperparameters()[0].values

{'units0': 112,
 'activation0': 'tanh',
 'num_layers': 1,
 'optimizer': 'sgd',
 'units1': 80,
 'activation1': 'sigmoid',
 'units2': 104,
 'activation2': 'sigmoid',
 'units3': 56,
 'activation3': 'tanh',
 'units4': 64,
 'activation4': 'relu',
 'units5': 8,
 'activation5': 'relu',
 'units6': 24,
 'activation6': 'tanh',
 'units7': 104,
 'activation7': 'sigmoid',
 'units8': 56,
 'activation8': 'sigmoid',
 'units9': 80,
 'activation9': 'sigmoid'}

In [48]:
tuned_model = tuner.get_best_models(num_models=1)[0]

In [49]:
tuned_model.fit(X_train,y_train,epochs=50, validation_data=(X_test,y_test))

Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0374 - loss: -88738.3828 - val_accuracy: 0.0382 - val_loss: -147657.7500
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0386 - loss: -163779.3750 - val_accuracy: 0.0382 - val_loss: -223611.8750
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0374 - loss: -238073.5469 - val_accuracy: 0.0382 - val_loss: -299566.0938
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.0387 - loss: -312353.8125 - val_accuracy: 0.0382 - val_loss: -375519.8750
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0414 - loss: -384452.8750 - val_accuracy: 0.0382 - val_loss: -451474.2812
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0389 - loss: -460440.1875 - val_accuracy: 0.0382 - val_loss: -527429.3750
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0388 - loss: -531515.5000 - val_accuracy: 0.0382 - val_loss: -603383.3750
Epoch 8/50
500/500 ━━

In [50]:
tuned_model.evaluate(X_train,y_train)

500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0380 - loss: -3801357.0000


[-3783095.25, 0.03831249848008156]